In this notebook we work on the problem of co-registering between CTX and CRISM sensors, in preparation for correcting image misalignment as part of the CaSSIS spectral parameters project.

We want the co-registered products to be in the same coordinate reference system (CRS), so we will also use the `pyproj` library to work with the data.

# Overview

1. Reading the CTX Dataset
2. Reading in the CRISM Dataset
3. Showing CTX/CRISM Misalignment
4. Co-registration with AROSICS
5. Apply Corrective Shift to other CRISM Products
6. Showing the Results

Please note that in addition to these Python packages, `ghostscript` version `9.56.1` is a requirement, to ensure PyGMT transparency works correctly (this may be an issue for MacOS only).

In [ ]:
# autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
import planetary_arosics as paros
from pathlib import Path
import pygmt
import rasterio as rio

# Reading in the CTX Dataset

In the `example_scenes` directory is a copy of the Jezero Crater orthorectified CTX 5 m/px resolution mosaic, that can be found here:

https://asc-pds-services.s3.us-west-2.amazonaws.com/mosaic/mars2020_trn/CTX/ScienceInvestigationMaps_JPL/M20_JezeroCrater_CTXortho_mosaic_5m.tif

The data is a GeoTIFF, with a coordinate reference system embedded in the file.

We read in this mosaic. This will be the target scene, that we will be co-registering other scenes to.

In [ ]:
ctx_mosaic_path = Path('.', 'input_products', 'ctx', 'orthomosaic', 'm20_jezerocrater', 'M20_JezeroCrater_CTXortho_mosaic_5m.tif')
ctx_mosaic_lbl = Path('.', 'input_products', 'ctx', 'orthomosaic', 'm20_jezerocrater', 'M20_JezeroCrater_CTXortho_mosaic_5m.xml')

The full mosaic, within it's coordinate reference system (CRS), covers the following region, that we display using PyGMT.

In [ ]:
def show_raster(raster_path: Path, title: str='') -> None:
    fig = pygmt.Figure()
    fig.grdimage(
        grid=raster_path,
        shading=False,
        cmap='gray',
        frame=['xaf','yaf', '+t'+title],
    )
    fig.show()

In [ ]:
show_raster(ctx_mosaic_path, 'Jezero Crater CTX Orthomosaic 5 m/px')

To align this CTX map with the CRISM map product, we need to convert the co-ordinate reference system to a geographic co-ordinate reference system, i.e. in units of Lat/Lon.

We achieve this using the following function, that accesses the geographic CRS (GRS) metadata, and performs the conversion.

We apply this to the CTX GeoTIFF. Note that this produces a new data product, that we attach the GRS suffix to, and save in the same directory as the source product.

In [ ]:
ctx_mosaic_grs_path = paros.geotiff_2_geo_crs(ctx_mosaic_path)

The product is now displayed with axes of Latitude and Longitude.

In [ ]:
show_raster(ctx_mosaic_grs_path, 'Jezero Crater CTX Orthomosaic 5 m/px')

In [ ]:
fig = pygmt.Figure()
fig.grdimage(
    grid=ctx_mosaic_grs_path,
    shading=False,
    cmap='gray',
    frame=['xaf','yaf', '+tJezero Crater CTX Orthomosaic 5 m/px'],
)
fig.show()

# Reading in the CRISM Data

Here we use the following CRISM scene to demonstrate the method.

https://ode.rsl.wustl.edu/mars/indexproductpage.aspx?product_idgeo=47360968&product_id=frt00005c5e_07_if166j_mtr3


We've downloaded the VNIR Apparent Reflectance (VNA) derived product to the example scenes directory of this repository. The VNA product shows surface brightness at 770 nm, and so is a good match to the 500 - 800 nm sensitivity of CTX.

More information on CRISM derived products can be found here: http://crism.jhuapl.edu/msl_landing_sites/index_news.php

The product is in PDS (Planetary Data System) `.img` format, so we will use the `pdr` Python package to read the dataset, and then convert this to a GeoTIFF for compatibility with the methods here.

In [ ]:
mtrdr_dir = Path('.', 'input_products', 'crism', 'mtrdr')
crism_vna_path = Path(mtrdr_dir, 'frt00005c5e_07_if166j_mtr3', 'frt00005c5e_07_brvnaj_mtr3').with_suffix('.img')

We've written the following function to translate a PDS data product to a GeoTIFF, preserving the data CRS information.

We apply this to the CRISM PDS product, noting that the GeoTIFF product will be saved next to the CRISM PDS product in the same directory.

In [ ]:
crism_vna_gtiff_path = paros.pds2geotiff(crism_vna_path)

We also convert this to a geographic Coordinate Reference System using the same function as before.

In [ ]:
crism_vna_grs_path = paros.geotiff_2_geo_crs(crism_vna_gtiff_path)

In [ ]:
crism_vna_grs = rio.open(crism_vna_grs_path)

In [ ]:
crism_vna_grs.count

In [ ]:
show_raster(crism_vna_grs_path, 'Source CRISM MTRDR VNA 18 m/px')

In [ ]:
fig = pygmt.Figure()
fig.grdimage(
    grid=crism_vna_grs_path,
    nan_transparent="white",
    cmap='gray',
    verbose=True,
    frame=['xaf','yaf', '+tSource CRISM MTRDR VNA 18 m/px'],
)
fig.show()

# Showing the Image Misalignment

We can focus on the Jezero Delta by providing an approximate bounding square in the default CRS.

We get this boundary out of the CRISM data product, by reading the GeoTIFF with `rasterio`.

In [ ]:
delta_wide_region = paros.get_raster_bounds(crism_vna_grs_path, str)

Now displaying this region, we can also overlay the CRISM product, with transparency to assess the mis-alignment.

This image is shown at the end of the notebook, o avoid a problem of crashing.

# Co-Registration with AROSICS

Here we attempt to interface with AROSICS directly, and do not use geowombat.

We have a problem where we need both datasets to use the same CRS.

Here we use the pyproj CRS library to take the CRS of the CTX basemap, and apply this to the CRISM image.

In [ ]:
crism_vna_grs_ctx_path = paros.inherit_crs(
                            ctx_mosaic_grs_path,
                            crism_vna_grs_path,
                            'ctx-crs'
                        )

Now we can check the CRS of the newly created geotiff of the CRISM scene.

In [ ]:
ctx_mosaic = rio.open(ctx_mosaic_grs_path)
crism_vna = rio.open(crism_vna_grs_ctx_path)
ctx_mosaic.crs == crism_vna.crs

## Global Co-Registration with AROSICS

First we will experiment with the simpler co-registration tool, COREG.

This finds an X/Y shift between a target and reference image.

Let's make a destination path for the co-registered CRISM image.

Now we pass the reference and target images to the COREG function.

For CRISM images, 255 is the no-data value that we set here.

We have found that a smaller window is more reliable, so we set this to 64 x 64 pixels.

This example image requires a shift greater than the default max_shift of 5. Here we set this to 20 pixels.

In [ ]:
ref_path = ctx_mosaic_grs_path
tgt_path = crism_vna_grs_ctx_path

In [ ]:
crism_vna_grs_ctx_coreg_path, CRG = paros.arosics_global_coreg(ref_path, tgt_path, no_data=255, ws=64, max_shift=20)

## Local Co-Registration with AROSICS

In [ ]:
ref_path = ctx_mosaic_grs_path
tgt_path = crism_vna_grs_ctx_coreg_path

In [ ]:
crism_vna_grs_ctx_local_coreg_path, CRL = paros.arosics_local_coreg(ref_path, tgt_path, grid_res=32, max_shift=20)

## Applying the Shifts to another CRISM Product

Let's now check that we can use the shifts found here to apply to a separate CRISM data product (derived from the same scene as the shifted product).

In [ ]:
mtrdr_dir = Path('.', 'input_products', 'crism', 'mtrdr')
crism_tru_path = Path(mtrdr_dir, 'frt00005c5e_07_if166j_mtr3', 'frt00005c5e_07_brtruj_mtr3').with_suffix('.img')

We also need to convert this product to the geographic coordinate reference system.

In [ ]:
crism_tru_gtiff_path = paros.pds2geotiff(crism_tru_path)
crism_tru_grs_ctx_path = paros.inherit_crs(
    ctx_mosaic_grs_path,
    crism_tru_gtiff_path,
    'ctx-crs'
)

In [ ]:
crism_tru_grs_ctx_path

In [ ]:
crism_tru_grs_ctx_crl_path = paros.apply_coreg(crism_tru_grs_ctx_path, global_crg=CRG, local_crg=CRL)

# Checking the results.

In [ ]:
def show_rasters(
        base_raster: Path, 
        layer_rasters: List[Path], 
        title: str='', 
        region: str=None) -> None:

    fig = pygmt.Figure()
    
    fig.grdimage(
        grid=base_raster,
        region=region,
        shading=False,
        cmap='gray',
        frame=['xaf','yaf', '+t'+title],
    )

    for raster_path in layer_rasters:
    
        fig.grdimage(
            grid=raster_path,
            region=region,
            shading=False,
            cmap='gray',
            nan_transparent="white",
            transparency=50,  # Set transparency for the drape grid
        )

    fig.show()

In [ ]:
fig = pygmt.Figure()
fig.grdimage(
    grid=ctx_mosaic_grs_path,
    region=delta_wide_region,
    shading=False,
    cmap='gray',
    frame=['xaf','yaf', '+tJezero Delta @^ CRISM MTRDR VNA 18 m/px with 5 m/px CTX @^ Prior to Coregistration'],
)

fig.grdimage(
    grid=crism_vna_grs_path,
    region=delta_wide_region,
    nan_transparent="white",
    transparency=50,  # Set transparency for the drape grid
)

fig.show()

In [ ]:
fig = pygmt.Figure()
fig.grdimage(
    grid=ctx_mosaic_grs_path,
    region=delta_wide_region,
    shading=False,
    cmap='gray',
    frame=['xaf','yaf', '+tJezero Delta @^ CRISM MTRDR VNA 18 m/px with 5 m/px CTX @^ Global Co-Registration'],
)

fig.grdimage(
    grid=crism_vna_grs_ctx_coreg_path,
    region=delta_wide_region,
    nan_transparent="white",
    verbose=True,
    transparency=50,  # Set transparency for the drape grid
)

fig.show()

In [ ]:
fig = pygmt.Figure()
fig.grdimage(
    grid=ctx_mosaic_grs_path,
    region=delta_wide_region,
    shading=False,
    cmap='gray',
    frame=['xaf','yaf', '+tJezero Delta @^ CRISM MTRDR VNA 18 m/px with 5 m/px CTX @^ Global & Local Co-Registration'],
)

fig.grdimage(
    grid=crism_vna_grs_ctx_local_coreg_path,
    region=delta_wide_region,
    nan_transparent="white",
    transparency=50,  # Set transparency for the drape grid
)

fig.show()

In [ ]:
fig = pygmt.Figure()
fig.grdimage(
    grid=ctx_mosaic_grs_path,
    region=delta_wide_region,
    shading=False,
    cmap='gray',
    frame=['xaf','yaf', '+tJezero Delta @^ CRISM MTRDR TRU 18 m/px with 5 m/px CTX @^ Global & Local Co-Registration'],
)

fig.grdimage(
    grid=crism_tru_grs_ctx_crl_path,
    region=delta_wide_region,
    nan_transparent="white",
    transparency=50,  # Set transparency for the drape grid
)

fig.show()

OK, this completes the goal of co-registration of geo-reference CRISM to CTX data.